In [1]:
!pip install open_clip_torch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0

In [6]:
import os
from PIL import Image
import torch
import torch.nn.functional as F
from transformers import BlipProcessor, BlipForConditionalGeneration
from transformers import CLIPProcessor, CLIPModel
from open_clip import create_model_from_pretrained, get_tokenizer

## BLIP

In [3]:
model_name = "Salesforce/blip-image-captioning-large"
processor = BlipProcessor.from_pretrained(model_name)
model = BlipForConditionalGeneration.from_pretrained(model_name)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [4]:
base = "/kaggle/input/cse344-a3/samples"
images = [{"name":img, "image":Image.open(os.path.join(base,img)).convert("RGB")} for img in os.listdir(base)]

In [8]:
for img_dict in images:
    inputs = processor(img_dict["image"], return_tensors="pt")
    out = model.generate(**inputs)
    img_dict["blip"] = processor.decode(out[0], skip_special_tokens=True)
    print(f"{img_dict['name']} : {img_dict['blip']}")

ILSVRC2012_test_00000004.jpg : arafed dog running in a field of grass with a sky background
ILSVRC2012_test_00000022.jpg : there is a small dog standing on a ledge near a pool
ILSVRC2012_test_00000023.jpg : they are riding on a bike in the rain in the street
ILSVRC2012_test_00000026.jpg : arafed man in a suit and tie sitting on a couch
ILSVRC2012_test_00000018.jpg : there are four children sitting on a towel by a pool
ILSVRC2012_test_00000003.jpg : araffe dog running on a leash at a dog show
ILSVRC2012_test_00000019.jpg : there is a bird that is sitting on a plant with green leaves
ILSVRC2012_test_00000030.jpg : ducks are standing in the water and one is drinking
ILSVRC2012_test_00000034.jpg : two cups of coffee being poured into a coffee machine
ILSVRC2012_test_00000025.jpg : there is a brown butterfly sitting on a leaf in the grass


## CLIP

In [9]:
clip_model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [16]:
for img_dict in images:
    inputs = clip_processor(text=img_dict["blip"], images=img_dict["image"], return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = clip_model(**inputs)
        logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    img_dict["clip"] = logits_per_image[0][0].item()
    print(f"{img_dict['name']} : {img_dict['blip']},  {img_dict['clip']:.4f}")

ILSVRC2012_test_00000004.jpg : arafed dog running in a field of grass with a sky background,  30.7687
ILSVRC2012_test_00000022.jpg : there is a small dog standing on a ledge near a pool,  30.2497
ILSVRC2012_test_00000023.jpg : they are riding on a bike in the rain in the street,  32.4249
ILSVRC2012_test_00000026.jpg : arafed man in a suit and tie sitting on a couch,  28.6873
ILSVRC2012_test_00000018.jpg : there are four children sitting on a towel by a pool,  33.6626
ILSVRC2012_test_00000003.jpg : araffe dog running on a leash at a dog show,  33.2162
ILSVRC2012_test_00000019.jpg : there is a bird that is sitting on a plant with green leaves,  26.5880
ILSVRC2012_test_00000030.jpg : ducks are standing in the water and one is drinking,  30.5419
ILSVRC2012_test_00000034.jpg : two cups of coffee being poured into a coffee machine,  30.2205
ILSVRC2012_test_00000025.jpg : there is a brown butterfly sitting on a leaf in the grass,  29.0515


## CLIPS

In [12]:
clips_model, clips_preprocess = create_model_from_pretrained('hf-hub:UCSC-VLAA/ViT-L-14-CLIPS-224-Recap-DataComp-1B')
clips_tokenizer = get_tokenizer('hf-hub:UCSC-VLAA/ViT-L-14-CLIPS-224-Recap-DataComp-1B')

open_clip_pytorch_model.bin:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
for img_dict in images:
    image = clips_preprocess(img_dict["image"]).unsqueeze(0)
    text = clips_tokenizer(img_dict["blip"], context_length=clips_model.context_length)
    
    with torch.no_grad(), torch.amp.autocast('cuda'):
        image_features = clips_model.encode_image(image)
        text_features = clips_model.encode_text(text)
        image_features = F.normalize(image_features, dim=-1)
        text_features = F.normalize(text_features, dim=-1)
        similarity_scores = (100.0 * image_features @ text_features.T) #unnormalized similarity scores

    img_dict["clips"] = similarity_scores[0][0].item()
    print(f"{img_dict['name']} : {img_dict['blip']},  {img_dict['clips']:.4f}")

ILSVRC2012_test_00000004.jpg : arafed dog running in a field of grass with a sky background,  15.5006
ILSVRC2012_test_00000022.jpg : there is a small dog standing on a ledge near a pool,  14.5663
ILSVRC2012_test_00000023.jpg : they are riding on a bike in the rain in the street,  17.3345
ILSVRC2012_test_00000026.jpg : arafed man in a suit and tie sitting on a couch,  13.4429
ILSVRC2012_test_00000018.jpg : there are four children sitting on a towel by a pool,  19.8995
ILSVRC2012_test_00000003.jpg : araffe dog running on a leash at a dog show,  17.5147
ILSVRC2012_test_00000019.jpg : there is a bird that is sitting on a plant with green leaves,  9.3397
ILSVRC2012_test_00000030.jpg : ducks are standing in the water and one is drinking,  15.8904
ILSVRC2012_test_00000034.jpg : two cups of coffee being poured into a coffee machine,  15.4525
ILSVRC2012_test_00000025.jpg : there is a brown butterfly sitting on a leaf in the grass,  12.2293
